In [9]:
# 1) Prepara datos (año, pivot por país y estándar de tecnologías)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]
for c in year_cols: df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
f = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()
YEAR = [c for c in year_cols if f[c].notna().any()][-1]

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

# Pivot por país (1 punto = país) para el scatter
idx = pd.MultiIndex.from_product([f["Country"].dropna().unique(),
                                  sorted(f["Technology_std"].dropna().unique())],
                                 names=["Country","Technology_std"])
grid = (f.groupby(["Country","Technology_std"])[YEAR].sum(min_count=1)
          .reindex(idx, fill_value=0.0).reset_index())
piv = (grid.pivot(index="Country", columns="Technology_std", values=YEAR)
          .fillna(0.0).reset_index())

for c in ["Solar","Wind","Hydro","Bio","Fossil"]:
    if c not in piv.columns: piv[c] = 0.0
piv["Total"] = piv.drop(columns=["Country"]).sum(axis=1, numeric_only=True)
piv["DominantTech"] = piv[["Solar","Wind","Hydro","Bio","Fossil"]].idxmax(axis=1)

# Para que no se amontonen, quitamos (0,0) solo del SCATTER:
piv_viz = piv[(piv["Solar"]>0) | (piv["Wind"]>0)].copy()
print(f"Año: {YEAR} | países con Solar/Wind>0: {len(piv_viz)}  | filas totales CSV: {len(f)}")

Año: F2023 | países con Solar/Wind>0: 82  | filas totales CSV: 570


In [10]:
from Isea import ScatterBrush
from IPython.display import display
from ipywidgets import Output
import pandas as pd

# sample data (note the actual column names)
df = pd.DataFrame({
    "id": [1,2,3,4],
    "x":  [1,2,3,4],
    "y":  [10,8,6,9],
    "group": ["A","B","A","B"]   # <-- categorical for color
})

# instantiate with the CORRECT column names
w = ScatterBrush(
    df,
    x="x", y="y",           # <-- must match df
    color="group",          # optional, only if it exists
    key="id",               # optional; defaults to a stable id anyway
    title="My Scatter", x_label="X", y_label="Y",
    width=720, height=420
)
display(w)

# show selection live below the chart
out = Output()
def on_sel(change):
    sel = change["new"]
    with out:
        out.clear_output()
        if sel.get("type"):
            rows = sel.get("rows", [])
            print(f"{sel['type']} selection — {len(rows)} points")
            display(pd.DataFrame(rows))
        else:
            print("Selection cleared")
w.observe(on_sel, names="selection")
display(out)


ScatterBrush(data=[{'id': 1, 'x': 1, 'y': 10, 'group': 'A'}, {'id': 2, 'x': 2, 'y': 8, 'group': 'B'}, {'id': 3…

Output()

In [11]:
display (out)

Output(outputs=({'name': 'stdout', 'text': 'set selection — 2 points\n', 'output_type': 'stream'}, {'output_ty…

In [3]:
import pandas as pd
from Isea import ParallelEnergy

df = pd.read_csv("Isea/Energy_clean.csv")

years = [c for c in df.columns if c.startswith("F")]
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])


In [4]:
w = ParallelEnergy(
    df, years,
    tech_col="Technology_std",
    label_col="Country",
    dims=("Fossil", "Solar", "Hydro", "Wind", "Bio"),
    year_start="F2023",
    width=1100, height=560,
    margin=dict(left=50, right=180, top=80, bottom=36),  # ajusta si quieres
    panel_position="bottom",       # "right" o "bottom"
    panel_width=320, panel_height=220,
    log_axes=False, normalize=False, reorder=True
)
w


ParallelEnergy(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008'…

In [6]:
w.show_selection()

,Country,Year,Fossil,Solar,Hydro,Wind,Bio,DominantTech
0,Australia,F2023,50820.000,32612.000,8439.960,12911.000,871.360,Fossil
1,Austria,F2023,4092.153,6831.704,14708.126,3977.236,1195.101,Hydro
2,Denmark,F2023,5255.738,3529.000,6.644,7481.818,2006.594,Wind
3,France,F2023,18499.104,20551.293,24139.006,20810.575,2187.252,Hydro
4,Germany,F2023,88934.000,81739.000,5741.000,69459.000,9950.000,Fossil
5,India,F2023,0.000,73109.342,47335.995,44736.240,10752.414,Solar
6,Italy,F2023,57284.374,29795.318,18847.790,12307.508,3434.457,Fossil
7,Japan,F2023,0.000,89077.000,28129.000,5343.000,5805.000,Solar
8,"Korea, Rep. of",F2023,90056.972,23493.085,1804.996,2204.860,2745.500,Fossil
9,Mexico,F2023,64909.500,10909.793,12838.009,7317.720,886.120,Fossil


,Country,Year,Fossil,Solar,Hydro,Wind,Bio,DominantTech
0,Australia,F2023,50820.000,32612.000,8439.960,12911.000,871.360,Fossil
1,Austria,F2023,4092.153,6831.704,14708.126,3977.236,1195.101,Hydro
2,Denmark,F2023,5255.738,3529.000,6.644,7481.818,2006.594,Wind
3,France,F2023,18499.104,20551.293,24139.006,20810.575,2187.252,Hydro
4,Germany,F2023,88934.000,81739.000,5741.000,69459.000,9950.000,Fossil
5,India,F2023,0.000,73109.342,47335.995,44736.240,10752.414,Solar
6,Italy,F2023,57284.374,29795.318,18847.790,12307.508,3434.457,Fossil
7,Japan,F2023,0.000,89077.000,28129.000,5343.000,5805.000,Solar
8,"Korea, Rep. of",F2023,90056.972,23493.085,1804.996,2204.860,2745.500,Fossil
9,Mexico,F2023,64909.500,10909.793,12838.009,7317.720,886.120,Fossil


In [7]:
from IPython.display import display, clear_output
import ipywidgets as ipw

out = ipw.Output()
display(out)

def on_sel(change):
    with out:
        clear_output(wait=True)
        df_sel = w.selection_df()
        print(f"point selection — {len(df_sel)} points")
        display(df_sel)

# Escucha cambios del widget
w.observe(on_sel, "selection")


Output()

In [8]:
w2 = w.new_from_selection(
    width=1000, height=520,
    margin=dict(l=48, r=160, t=72, b=32),
    panel_position="bottom",
    panel_height=200
)
w2


ParallelEnergy(data={'years': ['F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008'…